In [1]:
# [Cell 1] 기본 라이브러리 및 설정
import os
import json
import re
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# 설정값
INPUT_FILE = "project_full_context.txt"
OUTPUT_JSON = "project_flows.json"

# GPU 확인
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🚀 실행 환경: {'CUDA (GPU)' if torch.cuda.is_available() else 'CPU'}")

c:\Pyg\Projects\semi\yuzyproject-aimodels\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🚀 실행 환경: CUDA (GPU)


In [2]:
# [Cell 2] 초고속 경량 모델 로딩 (4비트 최적화)

# ======================================================
# [모델 선택] 아래 변수 중 하나만 주석(#)을 풀어서 사용.
# ======================================================

# 1. [추천] Microsoft Phi-3.5 (3.8B) - 4비트 시 속도 빠르고 지능 최강
MODEL_ID = "microsoft/Phi-3.5-mini-instruct"

# 2. [비상용] TinyLlama (1.1B) - 성능은 낮지만 압도적인 속도
# MODEL_ID = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# 3. [코드특화] StarCoder2 (3B) - 순수 코딩용 (채팅 능력은 부족할 수 있음)
# MODEL_ID = "bigcode/starcoder2-3b"

# 4. [안정성] Stable Code (3B) - Stability AI의 코딩 모델
# MODEL_ID = "stabilityai/stable-code-3b"

# ======================================================

# 4비트 양자화 설정 (속도 향상 및 메모리 절약 핵심)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

try:
    print(f"🔄 모델 로딩 중... ({MODEL_ID})")
    print("   -> 4비트 양자화로 로딩하여 속도를 높입니다.")
    
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
    
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        quantization_config=bnb_config, # 4비트 적용
        device_map="auto",
        trust_remote_code=True,
        attn_implementation="eager" # 에러 방지용
    )
    
    # 패딩 토큰 설정 (에러 방지)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        
    print("✅ 모델 로딩 완료!")

except Exception as e:
    print(f"❌ 모델 로딩 실패: {e}")

🔄 모델 로딩 중... (microsoft/Phi-3.5-mini-instruct)
   -> 4비트 양자화로 로딩하여 속도를 높입니다.


`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
Loading checkpoint shards: 100%|██████████| 2/2 [00:17<00:00,  8.55s/it]


✅ 모델 로딩 완료!


In [3]:
# [Cell 3] 프롬프트 매니저 (계층형 트리 구조 전용)

# 수정된 SYSTEM_PROMPT (DB 및 미들웨어 추적 강화)

SYSTEM_PROMPT = """
You are a 'Backend Code Analyst'.
Your task is to generate a **Precise Call Graph** based ONLY on the provided code.

### CRITICAL RULES (DO NOT HALLUCINATE)
1. **Identify Database Driver**: Check `package.json` or imports. Is it `mongoose`, `mysql2`, or `pg`?
   - If `mongoose` is used, DO NOT output SQL queries like `SELECT *`. Use Mongoose methods like `find()`, `save()`.
2. **Trace Middleware**: If a router has `isAuth` or `validate`, you MUST include them in `children`.
3. **Deep Trace**: Go from Router -> Middleware -> Controller -> Service/Data -> DB Library.

### JSON OUTPUT FORMAT
{
  "api": [
    {
      "category": "auth",
      "categoryName": "Auth Feature",
      "endpoints": [
        {
          "method": "POST",
          "url": "/auth/signup",
          "function": "signup",
          "file": "controller/auth.mjs",
          "description": "User Signup",
          "children": [
            {
              "function": "validateSignup",
              "file": "router/auth.mjs",
              "description": "Validation Chain",
              "children": []
            },
            {
              "function": "bcrypt.hashSync",
              "file": "controller/auth.mjs",
              "description": "Password Hashing",
              "children": []
            },
            {
              "function": "createUser",
              "file": "data/auth.mjs",
              "description": "Save to DB",
              "children": [
                {
                   "function": "User.save()",
                   "file": "mongoose",
                   "description": "MongoDB Insert",
                   "children": []
                }
              ]
            }
          ]
        }
      ]
    }
  ]
}
"""

print("✅ 계층형 분석 프롬프트 설정 완료")

✅ 계층형 분석 프롬프트 설정 완료


In [4]:
# [Cell 4] 코드 분석 및 JSON 생성 (경량 모델 최적화)

def extract_json(text):
    text = re.sub(r"^```(json)?", "", text.strip(), flags=re.MULTILINE)
    text = re.sub(r"```$", "", text.strip(), flags=re.MULTILINE)
    start = text.find('{')
    end = text.rfind('}')
    if start == -1 or end == -1: return "{}"
    return text[start:end+1]

def run_full_scan():
    if not os.path.exists(INPUT_FILE):
        print(f"❌ '{INPUT_FILE}' 파일이 없습니다.")
        return

    print(f"📂 '{INPUT_FILE}' 읽는 중...")
    with open(INPUT_FILE, 'r', encoding='utf-8') as f:
        full_code = f.read()
        
    # 경량 모델은 입력이 너무 길면 힘들어할 수 있어 30,000자로 제한 (필요시 조절)
    code_context = full_code[:50000] 

    # [중요] 경량 모델용 프롬프트 (명확하고 짧게 지시)
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": f"Analyze this code and return JSON ONLY:\n\n{code_context}"}
    ]

    # 채팅 템플릿 적용 (모델마다 다를 수 있어 안전장치 추가)
    try:
        text = tokenizer.apply_chat_template(
            messages, 
            tokenize=False, 
            add_generation_prompt=True
        )
    except Exception:
        # 템플릿 지원 안 하는 모델(StarCoder 등)을 위한 예외처리
        text = f"{SYSTEM_PROMPT}\n\nUser: Analyze this code:\n{code_context}\n\nAssistant:"

    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    print(f"🧠 {MODEL_ID.split('/')[-1]} 모델이 분석 중입니다...")

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=2048, # 경량 모델은 2048 정도가 안정적
        temperature=0.1,
        do_sample=True
    )

    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    # JSON 파싱 및 저장
    json_str = extract_json(response)
    
    try:
        data = json.loads(json_str)
        with open(OUTPUT_JSON, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        print(f"✅ JSON 생성 성공! -> {OUTPUT_JSON}")
        return data
    except json.JSONDecodeError as e:
        print("❌ JSON 파싱 실패. AI 응답을 확인하세요.")
        print("--- 응답 일부 ---")
        print(response[:500])
        return None

# 실행
json_data = run_full_scan()

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


📂 'project_full_context.txt' 읽는 중...
🧠 Phi-3.5-mini-instruct 모델이 분석 중입니다...


You are not running the flash-attention implementation, expect numerical differences.


✅ JSON 생성 성공! -> project_flows.json


In [5]:
# [Cell 5] 텍스트 트리 뷰어 (JSON 구조 확인용)

def print_tree(node, prefix="", is_last=True):
    """재귀적으로 JSON 트리를 출력"""
    connector = "└── " if is_last else "├── "
    
    # 노드 이름 결정 (function 또는 method+url)
    if "method" in node:
        name = f"[{node['method']}] {node['url']} ({node.get('description', '')})"
    elif "function" in node:
        name = f"ƒ {node['function']} - {node.get('description', '')}"
    elif "category" in node:
        name = f"📂 Category: {node.get('categoryName', node['category'])}"
    else:
        name = "Unknown Node"

    print(prefix + connector + name)
    
    # 자식 노드 순회
    children = node.get("children", [])
    if "endpoints" in node:
        children = node["endpoints"]
        
    count = len(children)
    for i, child in enumerate(children):
        new_prefix = prefix + ("    " if is_last else "│   ")
        print_tree(child, new_prefix, i == count - 1)

def visualize_json_structure():
    if not os.path.exists(OUTPUT_JSON):
        print("❌ JSON 파일이 없습니다.")
        return

    with open(OUTPUT_JSON, 'r', encoding='utf-8') as f:
        data = json.load(f)

    print("\n🌳 API Call Graph Analysis Result\n" + "="*40)
    if "api" in data:
        for cat in data["api"]:
            print_tree(cat)
    else:
        print("⚠️ 'api' 키를 찾을 수 없습니다. JSON 구조를 확인하세요.")

# 실행
visualize_json_structure()


🌳 API Call Graph Analysis Result
└── 📂 Category: Authentication Feature
    ├── [POST] /auth/signup (User Signup)
    │   ├── ƒ validateSignup - Validation Chain
    │   ├── ƒ bcrypt.hashSync - Password Hashing
    │   └── ƒ createUser - Save to DB
    │       └── ƒ User.save() - MongoDB Insert
    ├── [POST] /auth/login (User Login)
    └── [GET] /auth/me (Retrieve User Information)
└── 📂 Category: Post Management Feature
    ├── [GET] /posts (Retrieve All Posts)
    ├── [GET] /posts/:id (Retrieve Post by ID)
    ├── [POST] /posts (Create a New Post)
    │   └── ƒ validatePost - Request Validation
    ├── [PUT] /posts/:id (Update a Post)
    │   └── ƒ validatePost - Request Validation
    └── [DELETE] /posts/:id (Delete a Post)
        └── ƒ validatePost - Request Validation
